In [1]:
import json
import re
import os
import pandas as pd

from gensim.models import ldamodel
from gensim.corpora.dictionary import Dictionary
import pyLDAvis.gensim

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
stopwords = set(nltk.corpus.stopwords.words('english'))

data_path = '../data/'
model_path = '../model/'

def load_webhouse_data(data_name): 
    data_file = data_path + data_name + '.json'
    with open(data_file) as json_data:
        data = json.load(json_data)
        return data

def cleanup_text(text):
    text = re.sub(r'http\S+', '', text)
    text = text.replace("'s", " ")
    text = text.replace("n't", " not ")
    text = text.replace("'ve", " have ")
    text = text.replace("'re", " are ")
    text = text.replace("I'm"," I am ")
    text = text.replace("you're"," you are ")
    text = text.replace("You're"," You are ")
    text = text.replace("-"," ")
    text = text.replace("/"," ")
    text = text.replace("("," ")
    text = text.replace(")"," ")
    text = text.replace("%"," percent ")
    text = re.sub(r'[^a-zA-Z0-9 ]', '', text)
    text = " ".join([i for i in text.lower().split() if i not in stopwords])
    token = [WordNetLemmatizer().lemmatize(i) for i in text.split()]
    return token
    
# data = load_webhouse_data('microsoft')
data = load_webhouse_data('microsoft_0504_0604_clean') 

/usr/lib/python3/dist-packages/_pytest/fixtures.py:844: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  params = attr.ib(convert=attr.converters.optional(tuple))
/usr/lib/python3/dist-packages/_pytest/fixtures.py:846: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  ids = attr.ib(default=None, convert=_ensure_immutable_ids)


In [2]:
titles = [[data[i]['title'], data[i]['published'][:10]]  for i in range(len(data))]
df_feeds = pd.DataFrame(titles,columns=['title', 'date'])
titles = df_feeds[['title']].applymap(cleanup_text)['title']

In [6]:
dictionary = Dictionary(titles)
dictionary.filter_extremes(no_below=20, no_above=0.8)
corpora = [dictionary.doc2bow(doc) for doc in titles]

# Running and Trainign LDA model on the document term matrix.
lda_model = ldamodel.LdaModel(corpora, num_topics=7, id2word = dictionary, passes=50)

In [7]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpora, dictionary)
vis

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      15.956773        1       1  0.113433  0.241535
1      15.457237        1       2  0.123857 -0.123555
2      15.032985        1       3  0.151335 -0.048524
3      14.854714        1       4 -0.007242 -0.213229
6      14.687189        1       5 -0.190836  0.232901
4      12.810278        1       6 -0.336283 -0.106094
5      11.200825        1       7  0.145735  0.016967, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
18    Default   271.000000      window   271.000000  30.0000  30.0000
74    Default   275.000000      market   275.000000  29.0000  29.0000
21    Default   216.000000        game   216.000000  28.0000  28.0000
3     Default  1353.000000   microsoft  1353.000000  27.0000  27.0000
56    Default   179.000000        xbox   179.000000  26.0000  26.0000
8     Default   180.000000        tech   180.000000  25.0000  25.0000
14    Default   175.000000          10   175.000000  24.0000  24.0000
87    Default   166.000000          ai   166.000000  23.0000  23.0000
37    Default   210.000000       cloud   210.000000  22.0000  22.0000
168   Default   117.000000         day   117.000000  21.0000  21.0000
191   Default   134.000000      huawei   134.000000  20.0000  20.0000
27    Default   240.000000      google   240.000000  19.0000  19.0000
290   Default    97.000000       fox40    97.000000  18.0000  18.0000
25    Default   141.000000         may   141.000000  17.0000  17.0000
17    Default   117.000000      update   117.000000  16.0000  16.0000
99    Default   143.000000           u   143.000000  15.0000  15.0000
141   Default   101.000000      system   101.000000  14.0000  14.0000
125   Default    98.000000         say    98.000000  13.0000  13.0000
102   Default    90.000000         one    90.000000  12.0000  12.0000
26    Default   387.000000        2019   387.000000  11.0000  11.0000
69    Default    98.000000    software    98.000000  10.0000  10.0000
16    Default   385.000000         new   385.000000   9.0000   9.0000
175   Default    96.000000      global    96.000000   8.0000   8.0000
72    Default   132.000000    business   132.000000   7.0000   7.0000
178   Default    93.000000      launch    93.000000   6.0000   6.0000
184   Default    92.000000         ibm    92.000000   5.0000   5.0000
339   Default    81.000000          pc    81.000000   4.0000   4.0000
45    Default   113.000000        data   113.000000   3.0000   3.0000
30    Default   111.000000        live   111.000000   2.0000   2.0000
146   Default   156.000000       apple   156.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
298    Topic7    39.098671       storm    39.942051   2.1678  -3.9993
5      Topic7    37.606400     weekend    38.449478   2.1670  -4.0383
344    Topic7    36.434628      review    37.278111   2.1663  -4.0699
255    Topic7    36.016132       power    36.859810   2.1660  -4.0815
10     Topic7    34.807961  blockchain    35.651619   2.1652  -4.1156
215    Topic7    34.759205     project    35.602730   2.1652  -4.1170
258    Topic7    33.991104    national    34.834297   2.1647  -4.1393
230    Topic7    33.352715        work    34.196167   2.1642  -4.1583
127    Topic7    30.524235        rain    31.367325   2.1619  -4.2469
279    Topic7    30.354086       photo    31.197474   2.1618  -4.2525
169    Topic7    29.952845      mother    30.795765   2.1614  -4.2658
91     Topic7    29.880779       drive    30.724522   2.1613  -4.2682
291    Topic7    28.587732    memorial    29.430618   2.1601  -4.3125
197    Topic7    27.563253        post    28.406952   2.1590  -4.3489
88     Topic7    24.970203      annual    25.813633   2.1560  -4.4477
162    Topic7    24.877588        gift    25.720642   2.1559  -4.4515
179    Topic7    23.585527         set   

In [8]:
data[0]

{'author': 'editor',
 'crawled': '2019-05-05T00:21:17.002+03:00',
 'entities': {'locations': [],
  'organizations': [{'name': 'microsoft', 'sentiment': 'none'},
   {'name': 'npr', 'sentiment': 'none'},
   {'name': 'la clippers', 'sentiment': 'none'},
   {'name': 'getty images listening', 'sentiment': 'none'}],
  'persons': [{'name': 'steve ballmer', 'sentiment': 'none'},
   {'name': 'miguel vi', 'sentiment': 'none'},
   {'name': 'peter sagal', 'sentiment': 'none'},
   {'name': 'bill gates', 'sentiment': 'none'},
   {'name': 'ballmer', 'sentiment': 'none'}]},
 'external_images': [],
 'external_links': ['https://www.npr.org.', 'https://npr.org.'],
 'highlightText': '',
 'highlightTitle': '',
 'language': 'english',
 'ord_in_thread': 0,
 'published': '2019-05-04T18:02:00.000+03:00',
 'rating': None,
 'text': "Not My Job: We Quiz Former Microsoft CEO Steve Ballmer On Lip Balm By editor • 16 hours ago Miguel Villagran / Getty Images Listening... /\nLegend has it that when a young man named 